# Clinical outcome worked example

This notebook describes how the clinical outcome model works.

mRS: modified Rankin scale

Summary of the method:
1. Import the base mRS distributions and calculate their mean mRS and utilities
2. Gather the patient pathway data
3. Calculate the patient outcomes by stroke type and treatment
4. Combine the patient subgroups to match the input patient array

## Base outcomes and means

The outcome model uses the following usual probability distributions of modified Rankin scale (mRS):

![](clinical_outcome_worked_example/input_mrs_dists.png)

The extra blocks on the right-hand-side show the mean mRS across each distribution. The mRS distributions are also converted to utility-weighted distributions, and the mean of each of these is also shown on the right.

The mean mRS and mean utility values use these colours:

![](clinical_outcome_worked_example/colour_scales.png)

Incidentally the continuous mRS colours look minging and are given more for an idea of the mean mRS rather than seriously reading colours from this colour scale.

The conversion between mRS and utility is as follows:
| mRS $x$: | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 
| --- | --- | --- | --- | --- | --- | --- | --- |
| Utility $u(x)$: | 0.97 | 0.88 | 0.74 | 0.55 | 0.20 | -0.19 | 0.00 |

### Calculating mean mRS and utility

The mean mRS, $\bar{x}$, of a probability distribution is found by multiplying each mRS value by the probability of that mRS value in the distribution, and then summing all of those values:

$$\bar{x} = \sum\limits_{x=0}^{6} [P(\mathrm{mRS}=x) \times x]$$

For mRS between 0 and 6 this is written in full as:

\begin{align*}
\bar{x} = \{ & [P(\mathrm{mRS}=0) \times 0] + \\
& [P(\mathrm{mRS}=1) \times 1] + \\
& [P(\mathrm{mRS}=2) \times 2] + \\
& [P(\mathrm{mRS}=3) \times 3] + \\
& [P(\mathrm{mRS}=4) \times 4] + \\
& [P(\mathrm{mRS}=5) \times 5] + \\
& [P(\mathrm{mRS}=6) \times 6] \} 
\end{align*}

Mean utility $\bar{u}$ is calculated in a similar way, except the probability of each mRS value $x$ is multiplied by the utility $u$ of that mRS value from the table above:

$$\bar{u} = \sum\limits_{x=0}^{6} [P(\mathrm{mRS}=x) \times u(\mathrm{mRS}=x)]$$

In full:

\begin{align*}
\bar{u} = \{ & [P(\mathrm{mRS}=0) \times 0.97] + \\
& [P(\mathrm{mRS}=1) \times 0.88] + \\
& [P(\mathrm{mRS}=2) \times 0.74] + \\
& [P(\mathrm{mRS}=3) \times 0.55] + \\
& [P(\mathrm{mRS}=4) \times 0.20] + \\
& [P(\mathrm{mRS}=5) \times -0.19] + \\
& [P(\mathrm{mRS}=6) \times 0.00] \} 
\end{align*}

The values in the mRS probability distribution sum to exactly one so there is no need to further divide these means by the size of the population.

For example, the nLVO pre-stroke mRS distribution is:

| mRS<=0 | mRS<=1 | mRS<=2 | mRS<=3 | mRS<=4 | mRS<=5 | mRS<=6 | 
| --- | --- | --- | --- | --- | --- | --- |
| 0.582881 | 0.745419 | 0.848859 | 0.951082 | 0.993055 | 1.00 | 1.00 |


| mRS=0 | mRS=1 | mRS=2 | mRS=3 | mRS=4 | mRS=5 | mRS=6 | 
| --- | --- | --- | --- | --- | --- | --- |
| 0.582881 | 0.162538 | 0.10344 | 0.102223 | 0.041973 | 0.006945 | 0.0000 |


Mean mRS, $\bar{x}$:

\begin{align*}
\bar{x} = \{ & [0.58 \times 0] + \\
&[0.16 \times 1] + \\
& [0.10 \times 2] + \\
& [0.10 \times 3] + \\
& [0.04 \times 4] + \\
& [0.01 \times 5] + \\
& [0.00 \times 6] \} \\
\bar{x} = & 0.88
\end{align*}


Mean utility, $\bar{u}$:

\begin{align*}
\bar{u} = \{ & [0.58 \times 0.97] + \\
& [0.16 \times 0.88] + \\
& [0.10 \times 0.74] + \\
& [0.10 \times 0.55] + \\
& [0.04 \times 0.20] + \\
& [0.01 \times -0.19] + \\
& [0.00 \times 0.00] \} \\
\bar{u} = & 0.85
\end{align*}

### Notes

The pre-stroke and no-treatment distributions are identical for all LVO patients regardless of whether IVT and/or MT were considered.

The median (not mean!!) mRS is the value where cumulative probability is 0.5. 

### Code reference

Given a cumulative mRS probability distribution, the mean mRS and utility are calculated by:

## Example patient pathway results

The outcome model needs the following information with one value for each patient:

+ Stroke type (code, 0=other, 1=nLVO, 2=LVO)
+ Onset to needle time (minutes)
+ Whether IVT was chosen (True or False)
+ Onset to puncture time (minutes)
+ Whether MT was chosen (True or False)

These values can be generated from the `SSNAP_Pathway` class. This worked example uses 100 patients generated from that pathway.

Summary of the stroke types and treatments given:

![](clinical_outcome_worked_example/patient_input_stats.png)

## Calculate outcomes

The outcomes are calculated separately for the three categories (nLVO with IVT, LVO with IVT, and LVO with MT) because they use separate probability distributions that vary differently with time.

These results are calculated in this way for the following categories of patient:
+ nLVO treated with IVT  - in this example, contains 7 patients
+ LVO treated with IVT - in this example, contains 2 patients
+ LVO treated with MT - in this example, contains 3 patients

The quantities calculated are:
+ Each patient's post-stroke mRS distribution
+ Each patient's post-stroke mean mRS and utility
+ Each patient's mean shift in mRS and utility between the post-stroke and no-treatment distributions
+ The mean of all treated patients' mean shifts in mRS and utility

Also stored in the same place for easy reference are:
+ Mean mRS and utility for the no-treatment population
+ Mean mRS and utility for the population treated after the time of no effect

### Results for all patients with nLVO and treated with nLVO on time

For nLVO with IVT, the post-stroke probability distributions for the treated patients in this example are:

![](clinical_outcome_worked_example/mrs_dists_treated_nlvo_ivt.png)

### Results for all patients treated on time

The shifts in mRS and utility for all treated patients and for all stroke and treatment types are summarised on the following graphs. Each black arrow represents one patient, and the length of the arrow shows the shift between the no-treatment value and the mean value at the patient's treatment time.

![](clinical_outcome_worked_example/times_shifts_mrs_combo.png)
![](clinical_outcome_worked_example/times_shifts_utility_combo.png)



### Code reference:

These results are calculated in:
+ `calculate_outcomes_for_nlvo_ivt()`
+ `calculate_outcomes_for_lvo_ivt()`
+ `calculate_outcomes_for_lvo_mt()`

Each of these calculates the bulk of the useful results in the function `_create_mrs_utility_dict()`. 

In each case, the generated arrays still contain one value per patient in the whole cohort but invalid patients have values of `np.NaN` (Not A Number). These are patients with a different stroke type. Patients who use the no-treatment or no-effect distributions are given those values here instead of `np.NaN`.

## Combine results for patient array

The final step is to combine all of the separate patient subgroups into one big set of data that matches the input details. So if you provide the clinical outcome model with 100 patients' details, the 10th patient in your input list will have their results stored 10th in the output lists.

Various averages and statisics are calculated across this whole cohort dataset, but they may or may not be as useful as keeping the subgroups separate. These values are:
+ Mean post-stroke mRS
+ Mean shift from no-treatment mRS to post-stroke mRS
+ Mean post-stroke utility
+ Mean shift from no-treatment utility to post-stroke utility


Patients who do not receive treatment use the no-treatment distribution from the input mRS distributions. 
Patients who receive treatment after the no-effect time use the no-effect distribution from the input mRS distributions.

### Code reference

The three separate subgroups (nLVO and IVT, LVO and IVT, LVO and MT) are combined in `_calculate_patient_outcomes()`.

For each of these quantities...
+ Each patient's post-stroke mRS distribution
+ Each patient's post-stroke mean mRS
+ Each patient's shift in mRS
+ Each patient's post-stroke mean utility
+ Each patient's shift in utility

... a new array with size of the number of patients in the whole cohort is created. Then the values calculated earlier for each patient are placed into their allocated space, so that Patient X in the input arrays matches Patient X in the output. Any patients who were not treated receive either the values of the not-treated distributions or `np.NaN` (Not A Number) values if the stroke type is unknown. The `np.NaN` can be sidestepped in later calculations by using e.g. `np.nanmean()` instead of `np.mean()`.

## Basic usage

Assuming that `combo_trial_dict` contains arrays named the following...
+ `stroke_type_code` 
+ `onset_to_needle_mins` 
+ `ivt_chosen_bool`
+ `onset_to_puncture_mins`
+ `mt_chosen_bool`

... each with `number_of_patients` values of the expected data type and in the expected range.